In [1]:
# Install necessary libraries
!pip install pandas scikit-learn transformers torch datasets

import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score , precision_score, recall_score

# Step 1: Load and Preprocess Dataset
def load_data(file_path):
    """Load dataset from a CSV file and check for required columns."""
    data = pd.read_excel(file_path)
#     data = data.sample(frac=0.04, random_state=42)  # Random 40%

    if 'tweet_english' not in data.columns:
        raise KeyError("The dataset must contain a 'text' column with tweet data.")
    if 'label' not in data.columns:
        print("Warning: No 'label' column found. Assuming this is the unlabeled data.")
        data['label'] = None  # Add a label column with NaN values if missing
    return data


def preprocess_text(df):
    """Preprocess text data."""
    df['tweet_english'] = df['tweet_english'].str.lower()
    df['tweet_english'] = df['tweet_english'].str.replace(r"http\S+|www\S+|https\S+", '', regex=True)  # remove URLs
    df['tweet_english'] = df['tweet_english'].str.replace(r'\@\w+|\#', '', regex=True)  # remove mentions and hashtags
    df['tweet_english'] = df['tweet_english'].str.replace(r'[^A-Za-z\s]', '', regex=True)  # remove non-alphanumeric chars
    return df

# Step 2: Define Labeled and Unlabeled Dataset
from sklearn.model_selection import train_test_split

def split_data(data, label_column='label', labeled_size=0.2, unlabeled_size=0.6, test_size=0.2):
    """Split dataset into labeled, unlabeled, and test data."""
    # Ensure the sum of labeled_size, unlabeled_size, and test_size equals 1
    assert labeled_size + unlabeled_size + test_size == 1, "The sizes must sum to 1."
    
    # Treat all data as unlabeled, including the rows with labels
    
    
    # Split the data into train (80%) and test (20%) sets
    train_data, test_data = train_test_split(data, test_size=test_size, random_state=42)
    
    # Now, from the train_data (80%), we select 20% as labeled data
    labeled_data, unlabeled_data = train_test_split(train_data, test_size=0.75, random_state=42)
    
    return labeled_data,  test_data , unlabeled_data




# Step 3: Initialize Model
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

def tokenize(batch):
    # Convert each item in the 'tweets' list to string using list comprehension
    batch['tweet_english'] = [str(item) for item in batch['tweet_english']]  
    return tokenizer(batch['tweet_english'], padding='max_length', truncation=True, max_length=128)
# Step 4: Training Arguments
def setup_training_args():
    training_args = TrainingArguments(
        output_dir="./results",
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,
        weight_decay=0.01,
        evaluation_strategy="epoch"
    )
    return training_args

# Step 5: Initialize Trainer and Model Training
# ... (your existing code) ...

# Step 5: Initialize Trainer and Model Trainin
from datasets import Dataset  
def train_model(train_data, eval_data, model, training_args):
    # Convert DataFrames to Hugging Face Datasets
    train_dataset = Dataset.from_pandas(train_data.reset_index(drop=True))
    eval_dataset = Dataset.from_pandas(eval_data.reset_index(drop=True))
    
    # Tokenize datasets
    train_dataset = train_dataset.map(tokenize, batched=True)
    eval_dataset = eval_dataset.map(tokenize, batched=True)
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,  # Use Hugging Face Datasets
        eval_dataset=eval_dataset,    # Use Hugging Face Datasets
        compute_metrics=compute_metrics
    )
    trainer.train()
    return trainer

# ... (rest of your code) ...
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Step 6: Iterative Data Harvesting - Semi-Supervised Learning
# Step 6: Iterative Data Harvesting - Semi-Supervised Learning
def predict_with_confidence(trainer, data, threshold=0.9, batch_size=16):  # Added batch_size parameter
    """Predicts labels for unlabeled data with high confidence."""
    # Check if 'data' has any rows
    if data.empty:
        return pd.DataFrame(columns=data.columns)  # Return empty DataFrame

    # Convert each item in the 'tweets' list to string using list comprehension
    # and filter out any empty strings
    valid_tweets = [str(item) for item in data['tweet_english'].tolist() if str(item).strip()]
    
    # Check if there are any valid tweets
    if not valid_tweets:
        return pd.DataFrame(columns=data.columns)  # Return empty DataFrame
    
    # Process data in batches to reduce memory usage
    all_high_confidence_indices = []
    for i in range(0, len(valid_tweets), batch_size):
        batch_tweets = valid_tweets[i : i + batch_size]
        inputs = tokenizer(batch_tweets, return_tensors="pt", truncation=True, padding=True, max_length=128)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1)
        high_confidence_indices = (probs.max(dim=1).values > threshold).nonzero(as_tuple=True)[0]
        all_high_confidence_indices.extend(high_confidence_indices.cpu().numpy() + i)  # Adjust indices

    # If no high confidence predictions, return empty DataFrame
    if not all_high_confidence_indices:
        return pd.DataFrame(columns=data.columns)

    return data.iloc[all_high_confidence_indices]

def iterative_harvesting(trainer, labeled_data, unlabeled_data, iterations=1, threshold=0.9):
    for iteration in range(iterations):
        high_conf_data = predict_with_confidence(trainer, unlabeled_data, threshold)
        print(len(labeled_data))
        labeled_data = pd.concat([labeled_data, high_conf_data])
        print(len(labeled_data))
        unlabeled_data = unlabeled_data.drop(high_conf_data.index)
        trainer.train()  # Retrain on expanded labeled data
    return labeled_data, trainer

# Step 7: Zero-Shot Learning Integration (optional)
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
candidate_labels = ["depression", "no depression"]



# Step 8: Evaluation Metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = predictions.argmax(axis=1)  # Get the predicted class labels
    acc = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='binary')
    recall = recall_score(labels, preds, average='binary')
    f1 = f1_score(labels, preds, average='binary')
    
    return {'accuracy': acc, 'precision': precision, 'recall': recall, 'f1': f1}



# Main Pipeline Function
def evaluate_model(trainer, eval_data):
    # Re-tokenize eval_data to ensure consistency
#     print("a")
    eval_dataset = Dataset.from_pandas(eval_data.reset_index(drop=True))
#     print("b")
    eval_dataset = eval_dataset.map(tokenize, batched=True)
#     print("c")
    # Run evaluation
    final_results = trainer.evaluate(eval_dataset=eval_dataset)
#     print("s")
    return final_results

def run_pipeline(file_path):
    # Load and preprocess data
    data = load_data(file_path)
    data = preprocess_text(data)

    # Define labeled and unlabeled datasets
    train_data, eval_data, unlabeled_data = split_data(data)
    
    print(train_data)
    print(eval_data)
    print(unlabeled_data)
    # Initialize training arguments and trainer
    training_args = setup_training_args()
    print("training started.")
    trainer = train_model(train_data, eval_data, model, training_args)
    print("training ended.")
    # Iterative Data Harvesting
    print("Iterative Data Harvesting started.")
    final_labeled_data, final_trainer = iterative_harvesting(trainer, train_data, unlabeled_data)
    print("Iterative Data Harvesting ended.")

    # Final Evaluation
    print("Final Evaluation started.")
    
    final_results = evaluate_model(final_trainer, eval_data)
    print("Final model results:")
    print(f"Accuracy: {final_results['eval_accuracy']:.4f}")
    print(f"Precision: {final_results['eval_precision']:.4f}")
    print(f"Recall: {final_results['eval_recall']:.4f}")
    print(f"F1 Score: {final_results['eval_f1']:.4f}")
    return final_labeled_data, final_results

# Run the pipeline with your dataset file path
file_path = 'Arabic_Depression_10.000_Tweets_translated (2).xlsx'
run_pipeline(file_path)

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


/home/abcd/anaconda3/envs/abcd/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/home/abcd/anaconda3/envs/abcd/lib/python3.8/site-packages/transformers/utils/generic.py:482: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/abcd/anaconda3/envs/abcd/lib/python3.8/site-packages/transformers/utils/generic.py:339: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/abcd/anaconda3/envs/abcd/lib/python3.8/site-packages/transformers/utils/generic.py:339: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils

      Unnamed: 0                                              tweet  label  \
2130        2239  فترة طويلة كنت اعاني منه هلع و خوف من الموت فق...      1   
7974        3010                          هموت من الفرحه عشان علولو      0   
7306        2339    انا فرحان جدا برد فعل الناس على دور عم حسن ف...      0   
1182        1214                                    نوم واكل اكتئاب      1   
7370        2403                     انا فرحان جدا ل حوريه فرغلي ❤️      0   
...          ...                                                ...    ...   
8871        3910              هموت من الضحك المثل في محله تمام 😂😂😂😂      0   
9826        4865                ربنا يخلى الناس الحلوة فى حياتنا .       0   
5268         272  ربي اجعل اليوم أفضل من الأمس وغدًا أفضل من الي...      0   
9666        4705  نصيحة العمر  لن يتكرر  فلا تخسر وقتك بالحقد وا...      0   
6090        1116                                انا متفائل بكلامك 👍      0   

                                          tweet_english  
2130 

/home/abcd/anaconda3/envs/abcd/lib/python3.8/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.353677,0.956500,0.942584,0.973320,0.957705
2,0.006700,0.330137,0.961000,0.964215,0.958498,0.961348
3,0.006700,0.315376,0.962000,0.960630,0.964427,0.962525


Iterative Data Harvesting ended.
Final Evaluation started.


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Final model results:
Accuracy: 0.9620
Precision: 0.9606
Recall: 0.9644
F1 Score: 0.9625


(      Unnamed: 0                                              tweet  label  \
 2130        2239  فترة طويلة كنت اعاني منه هلع و خوف من الموت فق...      1   
 7974        3010                          هموت من الفرحه عشان علولو      0   
 7306        2339    انا فرحان جدا برد فعل الناس على دور عم حسن ف...      0   
 1182        1214                                    نوم واكل اكتئاب      1   
 7370        2403                     انا فرحان جدا ل حوريه فرغلي ❤️      0   
 ...          ...                                                ...    ...   
 9392        4431  بقالنا عشرات السنين بنتدرب على التأييد والشتيم...      0   
 904          926  أرى السحر طاقه شريره تظرب أماكن معينه بالدماغ ...      1   
 9133        4172                    محدش يقدر يشاركيني في رزقي    .      0   
 8855        3894               يقلع ابو الضحك يا ناس وشذا العقول         0   
 1972        2052  ايه واله وش ذا حزن بس ماعليه من التطبيق انت اف...      1   
 
                                           tweet_e